In [ ]:
## このプログラムは「Mercari Price Suggestion Challenge」において
## 作成したノートブックで動作します

import pandas as pd

# 訓練データテストデータをデータフレームに読み込む
train_df = pd.read_table('../input/mercari/train.tsv')
test_df = pd.read_table('../input/mercari/test.tsv')
print(train_df.shape, test_df.shape)

In [ ]:
#　訓練データの冒頭を表示
train_df.head()

In [ ]:
# 3ドル未満のレコードをすべて削除する
train_df = train_df.drop(train_df[(train_df.price < 3.0)].index)
print(train_df.shape)
print(train_df['price'].max())
print(train_df['price'].min())

In [ ]:
import matplotlib.pyplot as plt
train_df['price'].hist()  # 'price'のヒストグラム

In [ ]:
train_df['price'].hist(range=(0, 100)) # 範囲を3～100にする

In [ ]:
import numpy as np

# 訓練データのpriceを対数変換する
train_df["target"] = np.log1p(train_df.price)
# ヒストグラムを表示
train_df['target'].hist()

In [ ]:
# カテゴリ名を切り分けて新設のカラムに登録する

def split_cat(text):
    """
    ・カテゴリを/で切り分ける
    ・データが存在しない場合は'No Label'を返す
    """
    try: return text.split("/")
    except: return ('No Label', 'No Label', 'No Label')

# 3つに切り分けたカテゴリ名を'subcat_0'、'subcat_1'、'subcat_2'に登録
train_df['subcat_0'], train_df['subcat_1'], train_df['subcat_2'] = \
    zip(*train_df['category_name'].apply(lambda x: split_cat(x)))

test_df['subcat_0'], test_df['subcat_1'], test_df['subcat_2'] = \
    zip(*test_df['category_name'].apply(lambda x: split_cat(x)))

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
# 'brand_name'の対策

# train_dfとtest_dfを縦方向に結合
full_set = pd.concat([train_df, test_df])
# full_setの'brand_name'から重複なしのブランドリスト(集合)を生成
all_brands = set(full_set['brand_name'].values)

# 'brand_name'の欠損値NaNを'missing'に置き換える
train_df['brand_name'].fillna(value='missing', inplace=True)
test_df['brand_name'].fillna(value='missing', inplace=True)

# 訓練データの'brand_name'が'missing'に一致するレコード数を取得
train_premissing = len(train_df.loc[train_df['brand_name'] == 'missing'])
# テストデータの'brand_name'が'missing'に一致するレコード数を取得
test_premissing = len(test_df.loc[test_df['brand_name'] == 'missing'])

def brandfinder(line):
    """
    Parameters: line(str): ブランド名

    ・ブランド名の'missing'を商品名に置き換える:
         missing'の商品名の単語がブランドリストに存在する場合
    ・ブランド名を商品名に置き換える:
        商品名がブランドリストの名前と完全に一致する場合
    ・ブランド名をそのままにする:
        商品名がブランドリストの名前と一致しない
        商品名が'missing'だが商品名の単語がブランドリストにない
    """
    brand = line[0] # 第1要素はブランド名
    name = line[1]  # 第2要素は商品名
    namesplit = name.split(' ') # 商品名をスペースで切り分ける
    
    if brand == 'missing':  # ブランド名が'missing'と一致
        for x in namesplit: # 商品名から切り分けた単語を取り出す
            if x in all_brands:                
                return name # 単語がブランドリストに一致したら商品名を返す
    if name in all_brands:  # 商品名がブランドリストに存在すれば商品名を返す
        return name
    
    return brand            # どれにも一致しなければブランド名を返す

# ブランド名の付替えを実施
train_df['brand_name'] = train_df[['brand_name','name']].apply(brandfinder, axis = 1)
test_df['brand_name'] = test_df[['brand_name','name']].apply(brandfinder, axis = 1)

# 書き換えられた'missing'の数を取得
train_found = train_premissing-len(train_df.loc[train_df['brand_name'] == 'missing'])
test_found = test_premissing-len(test_df.loc[test_df['brand_name'] == 'missing'])
print(train_premissing) # 書き換える前の'missing'の数
print(train_found)      # 書き換えられた'missing'の数
print(test_premissing)  # 書き換える前の'missing'の数
print(test_found)       # 書き換えられた'missing'の数

In [ ]:
train_df.head()

In [ ]:
# 訓練データ、検証データ、テストデータを1つのデータフレームに連結
full_df = pd.concat([train_df, test_df], sort=False)

In [ ]:
# 連結データのカテゴリ名、ブランド名、説明文のNaNを'missing'に置き換える

def fill_missing_values(df):
    df.category_name.fillna(value='missing', inplace=True)    # カテゴリ名
    df.brand_name.fillna(value='missing', inplace=True)       # ブランド名
    df.item_description.fillna(value='missing', inplace=True) # 説明文
    # 説明文の'No description yet'を'missing'にする
    df.item_description.replace(
        'No description yet','missing', inplace=True) # 説明文の置き換え
    return df

full_df = fill_missing_values(full_df)

In [ ]:
# 連結データのカテゴリ、ブランド、3カテゴリのテキストをラベルエンコードする

from sklearn.preprocessing import LabelEncoder

# LabelEncoderの生成
le = LabelEncoder()

# 'category_name'をエンコードして'category'カラムに登録する
le.fit(full_df.category_name)
full_df['category'] = le.transform(full_df.category_name)

# 'brand_name'のエンコード
le.fit(full_df.brand_name)
full_df.brand_name = le.transform(full_df.brand_name)

# 'subcat_0'のエンコード
le.fit(full_df.subcat_0)
full_df.subcat_0 = le.transform(full_df.subcat_0)

# 'subcat_1'のエンコード
le.fit(full_df.subcat_1)
full_df.subcat_1 = le.transform(full_df.subcat_1)

# 'subcat_2'のエンコード
le.fit(full_df.subcat_2)
full_df.subcat_2 = le.transform(full_df.subcat_2)

del le

print(full_df.category.head())   # カテゴリ名
print(full_df.brand_name.head()) # ブランド名
print(full_df.subcat_0.head())   # 分割した1番目のカテゴリ
print(full_df.subcat_1.head())   # 分割した2番目のカテゴリ
print(full_df.subcat_2.head())   # 分割した3番目のカテゴリ

In [ ]:
# 連結データの説明文、商品名をラベルエンコードする

import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer

# 説明文、商品名、カテゴリ名の配列要素を以下のように1次元配列に連結する
# [[説明1,説明2, ..., 商品1,商品2, ..., カテゴリ1,カテゴリ2, ...]
#
print("Transforming text data to sequences...")
raw_text = np.hstack(
    [full_df.item_description.str.lower(), # 説明文
     full_df.name.str.lower(),             # 商品名
     full_df.category_name.str.lower()]    # カテゴリ名
)
print('sequences shape', raw_text.shape)

# 説明文、商品名、カテゴリ名を連結した配列でTokenizerを作る
print("   Fitting tokenizer...")
tok_raw = Tokenizer()
tok_raw.fit_on_texts(raw_text)

# Tokenizerで説明文、商品名をそれぞれラベルエンコードする
print("   Transforming text to sequences...")
full_df['seq_item_description'] = tok_raw.texts_to_sequences(full_df.item_description.str.lower())
full_df['seq_name'] = tok_raw.texts_to_sequences(full_df.name.str.lower())

del tok_raw

print(full_df.seq_item_description.head())
print(full_df.seq_name.head())

In [ ]:
# 0でパディングして配列の長さを揃える
from keras.preprocessing.sequence import pad_sequences
print(pad_sequences(full_df.seq_item_description, maxlen=80),'\n') # 商品説明
print(pad_sequences(full_df.seq_name, maxlen=10))                  # 商品名